In [1]:
import requests
import sqlite3
import datetime
import time

# ✅ Your Grok API Key
GROK_API_KEY = "আপনার API কী"

# ✅ Grok API এন্ডপয়েন্ট (Groq-এর API URL)
GROK_API_URL = "https://api.groq.com/openai/v1/chat/completions"  # Groq-এর সঠিক API URL

# ✅ Setup SQLite Database
conn = sqlite3.connect('writing_history.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS writings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        input_text TEXT,
        summary TEXT,
        rewritten_text TEXT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.commit()

# ✅ API কলের জন্য রিট্রাই মেকানিজম
def make_api_request(payload, max_retries=3, retry_delay=20):
    headers = {
        "Authorization": f"Bearer {GROK_API_KEY}",
        "Content-Type": "application/json"
    }
    for attempt in range(max_retries):
        response = requests.post(GROK_API_URL, headers=headers, json=payload)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:  # Too Many Requests
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(retry_delay)
        else:
            raise Exception(f"Error from Groq API: {response.status_code} - {response.text}")
    raise Exception(f"Failed after {max_retries} retries: Rate limit exceeded.")

# ✅ Summarizer Function (Groq API ব্যবহার করে)
def summarize_text(text):
    payload = {
        "model": "llama3-8b-8192",  # Groq-এর সমর্থিত মডেল
        "messages": [{"role": "user", "content": f"Summarize this: {text}"}],
        "temperature": 0.7
    }
    response = make_api_request(payload)
    return response.json()["choices"][0]["message"]["content"].strip()

# ✅ Rewriter Function (Groq API ব্যবহার করে)
def rewrite_text(text):
    payload = {
        "model": "llama3-8b-8192",  # Groq-এর সমর্থিত মডেল
        "messages": [{"role": "user", "content": f"Rewrite this to make it clearer and more professional:\n\n{text}"}],
        "temperature": 0.7
    }
    response = make_api_request(payload)
    return response.json()["choices"][0]["message"]["content"].strip()

# ✅ Basic Plagiarism Checker (No API)
common_phrases = [
    "Education is the key to success",
    "Climate change is a global concern",
    "Technology is advancing rapidly",
    "Health is wealth",
    "Time and tide wait for none"
]

def check_basic_plagiarism(text):
    matches = []
    for phrase in common_phrases:
        if phrase.lower() in text.lower():
            matches.append(phrase)
    if matches:
        return f"⚠️ Possible Plagiarism Detected in these lines:\n" + "\n".join(matches)
    else:
        return "✅ No common plagiarism detected."

# ✅ Save to Database
def save_to_db(original, summary, rewritten):
    cursor.execute('''
        INSERT INTO writings (input_text, summary, rewritten_text)
        VALUES (?, ?, ?)
    ''', (original, summary, rewritten))
    conn.commit()

# ✅ Show Writing History
def show_history():
    cursor.execute('SELECT * FROM writings ORDER BY timestamp DESC')
    rows = cursor.fetchall()
    for row in rows:
        print(f"\n📝 Original: {row[1]}\n📚 Summary: {row[2]}\n🔁 Rewritten: {row[3]}\n⏰ Time: {row[4]}\n")

# ✅ Main Assistant Function
def ai_writing_assistant():
    print("🧠 Welcome to AI Writing Assistant (Powered by Groq)!\n")
    input_text = input("✍️ Enter your paragraph:\n")

    print("\n🔍 Generating summary...")
    summary = summarize_text(input_text)

    print("\n🔁 Rewriting for clarity...")
    rewritten = rewrite_text(input_text)

    print("\n🕵️ Checking basic plagiarism...")
    plagiarism_result = check_basic_plagiarism(input_text)

    print("\n📚 Summary:\n", summary)
    print("\n🔁 Rewritten:\n", rewritten)
    print("\n⚠️ Plagiarism Report:\n", plagiarism_result)

    save_to_db(input_text, summary, rewritten)
    print("\n✅ Writing saved to your personal history.")

# ✅ Run the Assistant
ai_writing_assistant()

# ✅ Optional: Show all saved history
# show_history()

🧠 Welcome to AI Writing Assistant (Powered by Groq)!

✍️ Enter your paragraph:
Climate change is one of the biggest threats to our planet. The rising temperatures are causing glaciers to melt and sea levels to rise. 

🔍 Generating summary...

🔁 Rewriting for clarity...

🕵️ Checking basic plagiarism...

📚 Summary:
 Here is a summary:

Climate change is a significant threat to the Earth, causing temperatures to rise and leading to the melting of glaciers and an increase in sea levels.

🔁 Rewritten:
 Here is a rewritten version of the text:

Climate change poses a significant and far-reaching threat to our planet. The ongoing increase in global temperatures is precipitating the melting of glaciers and a subsequent rise in sea levels, leading to profound and potentially devastating consequences for the environment and human societies.

⚠️ Plagiarism Report:
 ✅ No common plagiarism detected.

✅ Writing saved to your personal history.
